In [ ]:
from autumn.infrastructure.remote import springboard
from autumn.projects.sm_covid2.common_school.runner_tools import (
    run_full_analysis, 
    print_continuous_status, 
    download_analysis,
    DEFAULT_RUN_CONFIG,
)

### Define task function

#### Standard config

In [ ]:
DEFAULT_RUN_CONFIG

In [ ]:
ISO3 = "FRA"
ANALYSIS = "main"

N_CHAINS = DEFAULT_RUN_CONFIG['N_CHAINS']
N_OPTI_SEARCHES = DEFAULT_RUN_CONFIG['N_OPTI_SEARCHES']
OPTI_BUDGET = DEFAULT_RUN_CONFIG['OPTI_BUDGET']

METROPOLIS_TUNE = DEFAULT_RUN_CONFIG['METROPOLIS_TUNE']
METROPOLIS_DRAWS = DEFAULT_RUN_CONFIG['METROPOLIS_DRAWS']
METROPOLIS_METHOD = DEFAULT_RUN_CONFIG['METROPOLIS_METHOD']

FULL_RUNS_SAMPLES = DEFAULT_RUN_CONFIG['FULL_RUNS_SAMPLES']
BURN_IN = DEFAULT_RUN_CONFIG['BURN_IN']

### Check that configuration is relevant

In [ ]:
assert (METROPOLIS_DRAWS - BURN_IN) * N_CHAINS >= FULL_RUNS_SAMPLES
assert METROPOLIS_METHOD in ("DEMetropolis", "DEMetropolisZ")

In [ ]:
def remote_full_analysis_task(bridge: springboard.task.TaskBridge, iso3: str = 'FRA', analysis: str = "main"):
    
    import multiprocessing as mp
    mp.set_start_method('forkserver')

    bridge.logger.info(f"Running full analysis for {iso3}. Analysis type: {analysis}.")

    idata, uncertainty_df, diff_quantiles_df = run_full_analysis(
        iso3,
        analysis=analysis, 
        opti_params={'n_searches': N_OPTI_SEARCHES, 'search_iterations': OPTI_BUDGET},
        mcmc_params={'draws': METROPOLIS_DRAWS, 'tune': METROPOLIS_TUNE, 'chains': N_CHAINS, 'method': METROPOLIS_METHOD},
        full_run_params={'samples': FULL_RUNS_SAMPLES, 'burn_in': BURN_IN},
        output_folder=bridge.out_path,
        logger=bridge.logger
    )
   
    bridge.logger.info("Full analysis complete")

In [ ]:
mspec = springboard.EC2MachineSpec(N_CHAINS, 4, "compute")
task_kwargs = {
    "iso3": ISO3,
    "analysis": ANALYSIS
}
tspec = springboard.TaskSpec(remote_full_analysis_task, task_kwargs)

In [ ]:
analysis_title = "single_start"
config_str = f"_{ANALYSIS}_LHS{N_OPTI_SEARCHES}_opt{OPTI_BUDGET}_mc{METROPOLIS_TUNE}n{METROPOLIS_DRAWS}"

run_path = springboard.launch.get_autumn_project_run_path("school_project", ISO3, analysis_title + config_str)
run_path

In [ ]:
runner = springboard.launch.launch_synced_autumn_task(tspec, mspec, run_path)

In [ ]:
runner.s3.get_status()

In [ ]:
print(runner.top("%MEM"))

In [ ]:
# wait function with status printing
# print_continuous_status(runner)

In [ ]:
# run_path = 'projects/school_project/FRA/2023-08-22T1004-single_start_main_LHS4_opt10000_mc2000n10000'

In [ ]:
download_analysis(run_path)